## Finding the most profitable towns in a CCAA given a max price

In [1]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from functions import *

#### Importing geo data from csv file

In [2]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

#### Getting data about properties for sale to query accordingly properties for rent 

Importing properties for sale csv

In [3]:
data_sale_properties = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/sale/2021.11.15_cataluña_90000_10.csv', engine = 'python')
data_sale_properties = data_sale_properties.loc[:, ~data_sale_properties.columns.str.contains('^Unnamed')]

In [4]:
def checking_nulls(df):
    # This function shows which columns have null values and returns a df with only nulls
    for c in df.columns:
        null_count = df[c].isnull().sum()
        if null_count > 0:
            print ("The column ", c, " has ", null_count, " null values")
    nulls = df[df.isna().any(axis=1)]
    return nulls.head(3)

In [5]:
data_sale_properties = data_sale_properties.dropna()

In [6]:
checking_nulls(data_sale_properties)

,name,floor,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift,province,ccaa


In [7]:
data_sale_properties.head(3)

,name,floor,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift,province,ccaa
1,Piso Carrer camí de la pietat. Piso economico...,first,Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.0,88700.0,0.0,no,3.0,"Piso de 50 m2, con comedor de 15 m2, cocina in...",https://www.habitaclia.com/comprar-piso-econom...,yes,barcelona,cataluña
3,Piso Carrer madrid. Piso en c/ madrid - badalona,first,Badalona,Llefià,badalona,50.0,2.0,1.0,1.0,87000.0,4500.0,no,7.0,"Descripción Práctico piso en Badalona, Barcel...",https://www.habitaclia.com/comprar-piso-en_c_m...,yes,barcelona,cataluña
4,Piso en Lloreda. Solo inversores. se vende l...,eighth,Badalona,Lloreda,badalona,74.0,3.0,1.0,432.0,32000.0,8000.0,no,2.0,SOLO INVERSORES. VIVIENDA NO DISPONIBLE PARA V...,https://www.habitaclia.com/comprar-piso-solo_i...,yes,barcelona,cataluña


#### Cleaning imported data

In [8]:
def convert_to_num(df, column):
    for i,value in enumerate(df[column]):
        df[column] = df[column].astype(int)

In [9]:
convert_to_num(data_sale_properties, column = 'm2')
convert_to_num(data_sale_properties, column = 'n_rooms')
convert_to_num(data_sale_properties, column = 'n_bath')

#### Exploring numerical features

In [10]:
features = data_sale_properties.filter(['m2','n_bath', 'n_rooms'], axis=1)

In [11]:
data_sale_properties.describe().T

,count,mean,std,min,25%,50%,75%,max
m2,612.0,67.580065,20.335849,25.0,52.0,65.0,80.0,140.0
n_rooms,612.0,2.398693,0.894353,1.0,2.0,2.0,3.0,5.0
n_bath,612.0,1.153595,0.378562,1.0,1.0,1.0,1.0,4.0
price_m2,612.0,290.076797,394.119602,1.0,1.0,1.0,742.0,997.0
price,612.0,74496.495098,13125.569163,21400.0,69000.0,78000.0,85000.0,89900.0
price_reduction,612.0,2642.815359,6347.756290,0.0,0.0,0.0,3000.0,58800.0
last_update,612.0,3.287582,5.126770,0.0,0.0,0.0,5.0,20.0


#### Getting the maximum price for the rent of the properties that we want to scrape

We get the 50% quantile price of the imported properties listed for sale and we extrapolate an expected rent price for those properties assuming a 12% rent. Since we want a maximum price, we use a generous profitability ratio (12%).

In [12]:
max_prices = [300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1500, 1800, 2000, 3000, 4000, 5000]

In [13]:
max_price = data_sale_properties.price.quantile(0.5)*0.12/12

if max_price < 1200:
    max_price = max_price/100
    max_price = math.ceil(max_price)*100
elif max_price > 1200 & max_price < 1500:
    max_price = 1500
elif max_price > 1500 & max_price < 1800:
    max_price = 1800
elif max_price > 1800 & max_price < 2000:
    max_price = 2000
elif max_price > 2000 & max_price < 3000:
    max_price = 3000
elif max_price > 3000 & max_price < 4000:
    max_price = 4000
elif max_price > 4000 & max_price < 5000:
    max_price = 5000
elif max_price > 5000:
    max_price = 5000

In [14]:
max_price

800

In [15]:
max_price = 1300

#### Get input from user

In [16]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

ccaa:  cataluña


In [17]:
num_properties_per_town = ""
while num_properties_per_town == "":
    try:
        num_properties_per_town = int(input("Minimum number of properties per town: "))
    except: 
        num_properties_per_town = ""

Minimum number of properties per town:  5


In [18]:
filtered_df = geo_data[(geo_data['ccaa'] == ccaa) & (geo_data['n_properties'] > num_properties_per_town)]

#### List of towns that we need to extract data from

In [19]:
towns = filtered_df[filtered_df['ccaa'] == ccaa].town.to_list()

Hay que hacer un for loop que para cada town saque el df y le haga un append a el global.

#### Extracting data from each town

In [20]:
property = "alquiler"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
maximum_price = "pmax="+str(max_price)
metros = "m2="+"50"

In [21]:
url = "https://www.habitaclia.com/"+property+ascensor+"-"+municipio+".htm?"+habitaciones+baños+maximum_price

In [22]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x] }
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,5))
            url = "https://www.habitaclia.com/"+property+"-"+t+"-"+str(p)+".htm?"+maximum_price
            #print(url)
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
            #print('town: ', t, " page: ", p)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties): 
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''

        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town] }
        df_town = pd.DataFrame.from_dict(dct_town)
        df = df.append(df_town, ignore_index = True)

    except:
        pass
        #print("no properties found at: ", url)

#### Output from the web scrapping

In [23]:
df.shape

(4926, 15)

In [24]:
df1 = df.copy()

#### Cleaning the df

In [25]:
df1 = df1.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1 = df1.drop(["town_x", "town_y", "n_properties"], axis=1)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [26]:
def clean_last_update():
    for i,n in enumerate(df1['last_update']):
        try:
            df1['last_update'][i] = df1['last_update'][i][0]
        except:
            df1['last_update'][i] = 'null'
clean_last_update()

In [27]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']
def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)
anytype_to_numerical(df1, columns = numericals)

In [28]:
df1.head(3)

,name,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,province,ccaa
0,Alquiler Piso Carrer d´avall. Piso nuevo muy ...,Arenys de Mar,Centre,arenys_de_mar,75,2,1,15,1150,0,yes,2,"Piso de obra nueva en Arenys de mar, cerca de ...",https://www.habitaclia.com/alquiler-piso-nuevo...,barcelona,cataluña
1,Alquiler Apartamento N ii junquera. Por meses...,Arenys de Mar,Urbanitzacions,arenys_de_mar,70,3,1,12,900,80,no,null,INMUEBLE NO SUJETO A LA LIMITACIÓN DE PRECIOS ...,https://www.habitaclia.com/alquiler-apartament...,barcelona,cataluña
2,"Alquiler Piso en Carrer mig (del), 20. Vivie...",Arenys de Mar,Urbanitzacions,arenys_de_mar,89,2,1,7,680,20,no,8,Vivienda en la Urbanización Carolines de Areny...,https://www.habitaclia.com/alquiler-piso-vivie...,barcelona,cataluña


#### Getting info about lifts

In [29]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []
lift = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x], 'lift':lift[:x]}
df_lift = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,5))
            url = "https://www.habitaclia.com/"+property+"-"+"viviendas-ascensor"+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):    
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = 'yes'

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)
                    lift.append(lift_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''
        
        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town], 'lift':lift[:x_town]}
        df_town = pd.DataFrame.from_dict(dct_town)
        df_lift = df_lift.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        pass
        #print("no properties found at: ", url)

In [30]:
df1_lift = df_lift.copy()

In [31]:
df1_lift = df1_lift.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1_lift = df1_lift.drop(["town_x", "town_y", "n_properties"], axis=1)
df1_lift = df1_lift.loc[:, ~df1_lift.columns.str.contains('^Unnamed')]

In [32]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1_lift['last_update']):
        try:
            df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
        except:
            df1_lift['last_update'][i] = '0'

In [33]:
anytype_to_numerical(df1_lift, columns = numericals)
clean_last_update()

In [34]:
df1_lift= df1_lift[df1_lift['price']<int(max_price)]

In [35]:
df1_lift.shape

(2696, 17)

In [36]:
list_names_with_lift = list(df1_lift['name'])

In [37]:
final_df = df1.combine_first(df1_lift)

In [38]:
final_df.shape

(4926, 17)

#### Data about the floor of each property

In [39]:
final_df.insert(1, 'floor', '')

In [40]:
final_df.head(3)

,area,floor,ccaa,description,geo_town,last_update,lift,m2,n_bath,n_rooms,name,neighborhood,opportunity,price,price_m2,price_reduction,province,url
0,Arenys de Mar,,cataluña,"Piso de obra nueva en Arenys de mar, cerca de ...",arenys_de_mar,2,yes,75,1,2,Alquiler Piso Carrer d´avall. Piso nuevo muy ...,Centre,yes,1150,15,0,barcelona,https://www.habitaclia.com/alquiler-piso-nuevo...
1,Arenys de Mar,,cataluña,INMUEBLE NO SUJETO A LA LIMITACIÓN DE PRECIOS ...,arenys_de_mar,null,yes,70,1,3,Alquiler Apartamento N ii junquera. Por meses...,Urbanitzacions,no,900,12,80,barcelona,https://www.habitaclia.com/alquiler-apartament...
2,Arenys de Mar,,cataluña,Vivienda en la Urbanización Carolines de Areny...,arenys_de_mar,8,yes,89,1,2,"Alquiler Piso en Carrer mig (del), 20. Vivie...",Urbanitzacions,no,680,7,20,barcelona,https://www.habitaclia.com/alquiler-piso-vivie...


In [41]:
for l in list_floors:
    find_floor(final_df, 
               column = 'description', 
               key_words = l)

In [42]:
get_lift_from_property_url(final_df)

0
200
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4300
4400
4500
4700
4800
4900


In [43]:
final_df1 = final_df.copy()

In [44]:
clean_floors2(final_df1, column = 'floor')

In [45]:
final_df1['floor'].value_counts()

first              683
second             550
third              487
fourth             282
sixth_or_higher    196
fifth              177
ground              56
underground          5
Name: floor, dtype: int64

In [46]:
final_df1['floor'].isnull().sum()

2490

In [47]:
final_df1['floor'].notnull().sum()

2436

Limpiar nuda propiedad

In [49]:
#Cleaning the final df
#df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]

#### Saving the scrapped df

In [48]:
today = datetime.now().strftime('%Y.%m.%d')
final_df1.to_csv(path_or_buf = '/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/rent/'+today+'_'+ccaa+'_'+str(max_price)+'_'+str(num_properties_per_town)+'.csv')